In [1]:
import lammps_logfile
import scipy
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import PySimpleGUI as sg
import numpy as np
import PySimpleGUI as sg
import matplotlib.widgets as wg
import matplotlib
import matplotlib.pyplot as plt
from ase.io.lammpsrun import read_lammps_dump_text

rdf_dict = {}
view_dict = {}

In [2]:
# Method used to load log files. It takes the path of the file and returns log and units.
# Log contains all the data and keywords that can be retrieved respectfully with
# log.get(keyword, simulation_number) and log.get_keywords(). Units contains unit types
# (e.g. 'metal') of simulations in file.
def read_log_file(file):
    log = lammps_logfile.File(file)
    units = []
    file.seek(0)
    for line in file:
        # Works for log format, usually a single simulation.
        if 'units' in line:
            line = line.partition(' ')[2]
            line = line.rstrip()
            units.append(line)
        # Works for out format, usually multiple simulations.
        if 'Unit style' in line:
            line = line.partition(': ')[2]
            line = line.rstrip()
            units.append(line)
    return log, units

In [3]:
# Meethod used to load rdf LAMMPS files.
def read_RDF_file(file, win_id):
    if win_id not in list(rdf_dict.keys()):
        rdf_dict[win_id] = {}
    data_dict = rdf_dict[win_id]
    try:
        x = []
        col_number = []
        missing_col_keys = []
        redundant_col_keys = []
        for line in file:
            if not line.startswith("#"):
                t = line.strip('\n').split()
                t = [float(x) for x in t]
                if len(t) == 2:
                    key, N = str(int(t[0])), t[1]
                else:
                    length = len(t)
                    col_number.append(length)
                    if length == col_number[0]:
                        x.append(t)
                    elif length < col_number[0]:
                        r = col_number[0]-length
                        missing_col_keys.append(key)
                        [t.append(0.0) for i in range(r)]
                        x.append(t)
                    else:
                        r = length-col_number[0]
                        redundant_col_keys.append(key)
                        [t.pop(-i) for i in range(1, r+1)]
                        x.append(t)

                if len(x) == N:
                    data_dict.update({key: x})
                    x = []

        if len(missing_col_keys) > 0:
            sg.popup_ok(
                f'A mismatch in number of columns occurred in timestep(s)\n{", ".join(missing_col_keys)}\nwhile attempting to load\n{file.name}.\nMissing rows were filled with 0.0.', title='Warning')
        if len(redundant_col_keys) > 0:
            sg.popup_ok(
                f'A mismatch in number of columns occurred in timestep(s)\n{", ".join(redundant_col_keys)}\nwhile attempting to load\n{file.name}.\nRedundant rows were removed.', title='Warning')

        rdf_dict[win_id] = data_dict
        return rdf_dict, col_number[0]
    except ValueError:
        sg.popup_ok(
            f'An error occurred while attempting to load\n{file.name}.', title='Error')
    except TypeError:
        sg.popup_ok(
            f'An error occurred while attempting to load\n{file.name}.', title='Error')

In [5]:
def read_dump_file(file, win_id):
    try:
        view_dict[win_id] = {}
        timesteps = []
        for line in file:
            if 'ITEM: TIMESTEP' in line:
                timesteps.append(file.readline().strip())

        for n in range(len(timesteps)):
            file.seek(0)
            key = timesteps[n]
            view_dict[win_id][key] = read_lammps_dump_text(file, index=n)
        number_of_atoms = len(view_dict[win_id][key])
        if number_of_atoms > 1000:
            sg.popup_ok(
                f'Dump file\n{file.name}\ncontains more than 1000 atoms. A smaller section of the simulation box will be shown fot the sake of clarity and performance.', title='Warning')
        return view_dict
    except ValueError:
        sg.popup_ok(
            f'An error occurred while attempting to load\n{file.name}.', title='Error')
    except TypeError:
        sg.popup_ok(
            f'An error occurred while attempting to load\n{file.name}.', title='Error')

['0', '1']


In [4]:
def linear_fit(x, y, lower_bound_id=0, higher_bound_id=0):
    if higher_bound_id != 0 and higher_bound_id > lower_bound_id:
        if higher_bound_id > x.size:
            higher_bound_id = x.size
        x_new = x[lower_bound_id:higher_bound_id]
        y_new = y[lower_bound_id:higher_bound_id]
    else:
        x_new = x
        y_new = y
    a, b, c, d, e = scipy.stats.linregress(x_new, y_new)

    return a, b

In [5]:
units_dict = {'metal': {'Step': '', 'Time': 'ps', 'TotEng': 'eV', 'Temp': 'K', 'Volume': 'A^3', 'Density': 'g/cm^3', 'Press': 'bar', 'Ndanger': '', 'PotEng': 'eV', 'KinEng': 'eV', 'Enthalpy': 'eV'},
              'real': {'Step': '', 'Time': 'fs', 'TotEng': 'kcal/mol', 'Temp': 'K', 'Volume': 'A^3', 'Density': 'g/cm^3', 'Press': 'atm', 'Ndanger': '', 'PotEng': 'kcal/mol', 'KinEng': 'kcal/mol', 'Enthalpy': 'kcal/mol'},
              'SI': {'Step': '', 'Time': 's', 'TotEng': 'J', 'Temp': 'K', 'Volume': 'm^3', 'Density': 'kg/m^3', 'Press': 'Pa', 'Ndanger': '', 'PotEng': 'J', 'KinEng': 'J', 'Enthalpy': 'J'},
              'LJ': {'Step': '', 'Time': '', 'TotEng': '', 'Temp': '', 'Volume': '', 'Density': '', 'Press': '', 'Ndanger': '', 'PotEng': '', 'KinEng': '', 'Enthalpy': ''}}


def units(units_type, x):
    return units_dict[units_type][x]

In [13]:
# Formats a number to sig significant figures, default 4
def format_number(number, sig=4):
    str_number = str(number)
    str_number = str_number.split('.')
    zeros = 0
    for char in list(str_number[1]):
        if str_number[0] == '0' and char == '0':
            zeros+=1
    n = sig + zeros
    formatted_number = format(number, f'.{n}f')
    return float(formatted_number)

0.006175

In [7]:
def intersection(a1, b1, a2, b2):
    x_intersection = (b2 - b1) / (a1 - a2)
    y_intersection = a1 * x_intersection + b1
    return x_intersection, y_intersection

In [8]:
def find_closest_point(x, y, x0, y0):
    min_distance = float('inf')
    closest_point_index = -1
    x_norm = abs(np.max(x)-np.min(x))
    y_norm = abs(np.max(y)-np.min(y))
    x00 = x0 / x_norm
    y00 = y0 / y_norm

    for i in range(len(x)):
        xp = x[i] / x_norm
        yp = y[i] / y_norm
        distance = np.sqrt((xp - x00)**2 + (yp - y00)**2)

        if distance < min_distance:
            min_distance = distance
            closest_point_index = i

    x_res = x[closest_point_index]
    y_res = y[closest_point_index]

    return x_res, y_res

In [9]:
def find_nearest_value(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [ ]:
def update_headings(table, headings):
    COL_HEADINGS = ['Line', 'Slope', 'Intercept', 'Boundry 1', 'Boundry 2']
    for cid, text in zip(COL_HEADINGS, headings):
        table.heading(cid, text=text)

In [10]:
# Method used to update comboboxes in main window upon loading new file.
def update_combos_main(keywords, n, window, tab_id):
    combo_n = np.arange(1, n+1).tolist()
    for i in range(2, 5):
        window[f'combo_n{i}_{tab_id}'].update(
            value=combo_n[0], values=combo_n)
        window[f'combo_x{i}_{tab_id}'].update(
            value=keywords[0], values=keywords)
        window[f'combo_y{i}_{tab_id}'].update(
            value=keywords[1], values=keywords)

    if 'Temp' and 'TotEng' in keywords:
        window[f'combo_n1_{tab_id}'].update(
            value=combo_n[0], values=combo_n)
        window[f'combo_x1_{tab_id}'].update(
            value='Temp', values=keywords)
        window[f'combo_y1_{tab_id}'].update(
            value='TotEng', values=keywords)

    else:
        window[f'combo_n1_{tab_id}'].update(
            value=combo_n[0], values=combo_n)
        window[f'combo_x1_{tab_id}'].update(
            value=keywords[0], values=keywords)
        window[f'combo_y1_{tab_id}'].update(
            value=keywords[1], values=keywords)

In [ ]:
# Method used to update comboboxes in analyze windows.
def update_combos_analyze(analyze_window, values, tab_id, combo_xy, n):
    combo_n = np.arange(1, n+1).tolist()

    for i in ('fit', 'rdf', 'view'):
        analyze_window[f'combo_x{i}_{tab_id}'].update(
            value=values[f'combo_x1_{tab_id}'], values=combo_xy)

        analyze_window[f'combo_y{i}_{tab_id}'].update(
            value=values[f'combo_y1_{tab_id}'], values=combo_xy)

        analyze_window[f'combo_n{i}_{tab_id}'].update(
            value=values[f'combo_n1_{tab_id}'], values=combo_n)